### Identify missing files

list missing files before processing

In [5]:
#setup libraries
import numpy as np
import pandas as pd
import os, re, glob, shutil, sys

In [6]:
directory_name = "AU565_drug_screen"
directory_name = "HCC1143_drug_screen"
directory_name = "21MT1_drug_screen"
#directory_name = "MDAMB157_drug_screen"
#directory_name = "AU_I_L_006_01_1"
#directory_name = "nlc_incucyte"


#directory_name = "AU565_Drug_Screen_Project" #used for files stored on graylab_share
plateID = "AU00601"
plateID = "HC01401"
plateID = "MD00601"
plateID = "2101301"
#plateID = "221006_nlc_hcc1143ccrep_sirna_ptx_p2"

def get_file_df(plateID):
    print("getting files for "+plateID)
    #data_path =  '/home/groups/graylab_share/OMERO.rdsStore/liby/'+directory_name+'/'+plateID+"/"+plateID+'_IMAGES'
    data_path =  '/home/groups/heiserlab_genomics/home/liby/'+directory_name+'/'+plateID+"/"+plateID+'_IMAGES'
    #data_path =  '/home/groups/heiserlab_genomics/home/liby/'+directory_name+'/'+plateID+'_IMAGES'
    #data_path = "/home/groups/heiserlab_genomics/home/grossse/AU565 Project/AU565 Image Files/AU_I_L_006_01_1/AU_I_L_006_01_1_IMAGES"
    data_paths = glob.glob(os.path.join(data_path,"*.tif"),recursive=False)
    df = pd.DataFrame(data_paths, columns =['path'])
    df['filename'] = df['path'].str.extract('(\w*.tif)')
    df['plateID'] = df['filename'].str.extract('(^[a-zA-Z0-9]*)')
    df['channel'] = df['filename'].str.extract('(_[PRG]_)')
    df['channel'] = df['channel'].str.extract('([PRG])')
    df['well'] = df['filename'].str.extract('(_[A-Z][0-9]+_)')
    df['well'] = df['well'].str.extract('([A-Z][0-9]+)')
    df['field'] = df['filename'].str.extract('(_[0-9]+_)')
    df['field'] = df['field'].str.extract('([0-9]+)')
    df['time'] = df['filename'].str.extract('(_[a-z0-9]*.tif)')
    df['time'] = df['time'].str.replace('[_(.tif)]','',regex = True)
    return df

        
dest_path = '/home/exacloud/gscratch/HeiserLab/images/'
data_path =  '/home/groups/heiserlab_genomics/home/liby/'+directory_name+'/'+plateID+"/"+plateID+'_IMAGES/'
#data_path =   '/home/groups/heiserlab_genomics/globus/calistri/'+directory_name+'/'+plateID


print("get file names in "+dest_path+plateID)
df_dest = get_file_df(plateID)
df_dest['dest_path'] = df_dest['plateID']+'/'+df_dest['well']+'/'+'field_'+df_dest['field']

channels = np.unique(df_dest.channel)
wells = np.unique(df_dest.well)
fields = np.unique(df_dest.field)
times = np.unique(df_dest.time)

fullset = []
for channel in channels:
    for well in wells:
        for field in fields:
            for time in times:
                fullset.append(data_path+plateID+"_"+channel+"_"+well+"_"+field+"_"+time+".tif")
                
df_missing_from_source = pd.DataFrame(set(fullset)-set(df_dest.path), columns =['path'])
df_missing_from_source['filename'] = df_missing_from_source['path'].str.extract('(\w*.tif)')

df_missing_from_source['plateID'] = df_missing_from_source['filename'].str.extract('(^[a-zA-Z0-9]*)')
df_missing_from_source['channel'] = df_missing_from_source['filename'].str.extract('(_[PRG]_)')
df_missing_from_source['channel'] = df_missing_from_source['channel'].str.extract('([PRG])')
df_missing_from_source['well'] = df_missing_from_source['filename'].str.extract('(_[A-Z][0-9]+_)')
df_missing_from_source['well'] = df_missing_from_source['well'].str.extract('([A-Z][0-9]+)')
df_missing_from_source['field'] = df_missing_from_source['filename'].str.extract('(_[0-9]+_)')
df_missing_from_source['field'] = df_missing_from_source['field'].str.extract('([0-9]+)')
df_missing_from_source['time'] = df_missing_from_source['filename'].str.extract('(_[a-z0-9]*.tif)')
df_missing_from_source['time'] = df_missing_from_source['time'].str.replace('[_(.tif)]','',regex = True)
time_slices = ['04d00h31m','04d01h01m','04d00h30m', '04d01h00m']
# filter out time slices after 4 days
df_missing_from_source = df_missing_from_source.loc[~df_missing_from_source['time'].isin(time_slices)].sort_values(by = ['channel', 'well', 'field', 'time'])
df_missing_from_source.reset_index(drop=True, inplace=True)

df_missing_from_source.to_csv(dest_path+plateID+"/"+plateID+"_missing_files_from_incucyte.csv")


get file names in /home/exacloud/gscratch/HeiserLab/images/2101301
getting files for 2101301


In [7]:
pd.set_option('display.max_colwidth', 200)  # or 199
df_missing_from_source

,path,filename,plateID,channel,well,field,time
